# Tutorial
Tutorial how to deploy an app since compute engine GCP

This is the oficial tutorial of GCP:
https://cloud.google.com/python/docs/getting-started/getting-started-on-compute-engine?hl=es-419

In the tutorial are executed a list of command in console but in this example a jupyter notebook is used to execute the commands

In [ ]:
import numpy as np

### 0) Project

In [ ]:
PROJECT_GCP = 'input-project-gcp...'

### 1) En Cloud Shell, configura la CLI de gcloud para usar el proyecto de Google Cloud nuevo de la siguiente manera

In [ ]:
# Configure gcloud for your project
!gcloud config set project $PROJECT_GCP

### 2) Usa una secuencia de comandos de inicio para inicializar una instancia
Necesitas una forma de indicar a tu instancia que descargue y ejecute tu código. Una instancia puede tener una secuencia de comandos de inicio que se ejecuta cada vez que se inicia o reinicia la instancia.

In [ ]:
# Install or update needed software
apt-get update
apt-get install -yq git supervisor python python-pip python3-distutils
pip install --upgrade pip virtualenv

# Fetch source code
export HOME=/root
git clone https://github.com/GoogleCloudPlatform/getting-started-python.git /opt/app

# Install Cloud Ops Agent
sudo bash /opt/app/gce/add-google-cloud-ops-agent-repo.sh --also-install

# Account to own server process
useradd -m -d /home/pythonapp pythonapp

# Python environment setup
virtualenv -p python3 /opt/app/gce/env
/bin/bash -c "source /opt/app/gce/env/bin/activate"
/opt/app/gce/env/bin/pip install -r /opt/app/gce/requirements.txt

# Set ownership to newly created account
chown -R pythonapp:pythonapp /opt/app

# Put supervisor configuration in proper place
cp /opt/app/gce/python-app.conf /etc/supervisor/conf.d/python-app.conf

# Start service via supervisorctl
supervisorctl reread
supervisorctl update

La secuencia de comandos de inicio realiza las siguientes tareas:

Instala Python 3 y PIP.

Instala el agente de Cloud Logging. Este agente recopila registros de syslog automáticamente.

Instala Supervisor para ejecutar la app como un daemon.

Clona el código fuente de la app desde Cloud Source Repositories y, luego, instala dependencias.

Configura Supervisor para ejecutar la app. Supervisor se asegura de que la app se reinicie si se cierra de forma inesperada o si la cierra un administrador o algún otro proceso. También envía stdout y stderr de la app a syslog para que los recopile el agente de Logging.

### 3. Crea y configura una instancia de Compute Engine

In [ ]:
MY_INSTANCE_NAME="my-app-instance"
ZONE="us-central1-a"

!gcloud compute instances create $MY_INSTANCE_NAME \
    --image-family=debian-10 \
    --image-project=debian-cloud \
    --machine-type=g1-small \
    --scopes userinfo-email,cloud-platform \
    --metadata-from-file startup-script=startup-script.sh \
    --zone $ZONE \
    --tags http-server

#### Ahora está creada la instancia, pero aún no se puede ver desde el navegador

### 4. Crea una regla de firewall para permitir el tráfico hacia tu instancia de la manera siguiente:

In [ ]:
!gcloud compute firewall-rules create default-allow-http-8080 \
    --allow tcp:8080 \
    --source-ranges 0.0.0.0/0 \
    --target-tags http-server \
    --description "Allow port 8080 access to http-server"

In [ ]:
##### PORT FOR ANALYTICS ENVIRONEMNT
!gcloud compute firewall-rules create default-allow-http-8888 \
    --allow tcp:8888 \
    --source-ranges 0.0.0.0/0 \
    --target-tags http-server \
    --description "Allow port 8080 access to http-server"

### 5. Obtén la dirección IP externa de la instancia

In [1]:
# !gcloud compute instances list

In [2]:
a=1
a

1

### 6. See the app

Ver desde el link
http://YOUR_INSTANCE_IP:8080

Reemplaza YOUR_INSTANCE_IP por la dirección IP externa de la instancia.

In [ ]:
http://YOUR_INSTANCE_IP:8080

Extra habilitar puerto streamlit

In [ ]:
!gcloud compute firewall-rules create default-allow-http-8501 \
    --allow tcp:8501 \
    --source-ranges 0.0.0.0/0 \
    --target-tags http-server \
    --description "Allow port 8501 access to http-server streamlit"

La secuencia de comandos de inicio realiza las siguientes tareas:

Instala Python 3 y PIP.

Instala el agente de Cloud Logging. Este agente recopila registros de syslog automáticamente.

Instala Supervisor para ejecutar la app como un daemon.

Clona el código fuente de la app desde Cloud Source Repositories y, luego, instala dependencias.

Configura Supervisor para ejecutar la app. Supervisor se asegura de que la app se reinicie si se cierra de forma inesperada o si la cierra un administrador o algún otro proceso. También envía stdout y stderr de la app a syslog para que los recopile el agente de Logging.